In [1]:
from bs4 import BeautifulSoup
import requests
import pandas
import re
import execjs
print("https://www.haozu.com/sh/house-list/")
print("note:\n如果unitPrice==0，则totalPrice的单位为 元/间/月")
prefix = 'https://www.haozu.com/sh/house-list/'
allUrl = ['https://www.haozu.com/sh/house-list/']
for item in range(1, 1) : # 循环定义所需要爬取的地址
    if item != 1 :
        a = str(prefix) + 'o' + str(item)
        allUrl.append(a)
info = [] # 定义一个列表，用来装载字典,为了方便后面的数据整理
rule = re.compile("\d+\.?\d*") # define a regular of getting number from string

for url in allUrl : #loop url
    headers={
            'Connection': 'Keep-Alive',
            'Accept': 'text/html, application/xhtml+xml, */*',
            'Accept-Language': 'en-US,en;q=0.8,zh-Hans-CN;q=0.5,zh-Hans;q=0.3',
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko'
        }
    res = requests.get(url,headers)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text,'html.parser')
    content = soup.select('.list-content')
    if len(content) == 0:  
        break
    for item in content:
        title = item.select('.h1-title a')[0].text
        
        area = item.select('.p1')[0].text.split('/')[1]
        area = re.search(rule,area)[0]
        
        totalPrice = item.select('.s1')[0].text 
        totalPrice = re.search(rule, totalPrice)
        totalPrice = float(totalPrice[0]) if totalPrice else 0
        totalPrice = totalPrice if totalPrice > 100 else totalPrice * 1000
         
        loc = item.select('.p1')[1].text.replace(' ','').replace('\n','')
        
        unitPrice = item.select('.s2')
        unitPrice = item.select('.s2')[0].text if unitPrice else '0'
        unitPrice = re.search(rule, unitPrice)
        unitPrice = float(unitPrice[0]) if unitPrice else 0

        # 获取详细信息
        href = item.select('a[style=""]')[0]['href']
        href = 'https://www.haozu.com' + href
        
        res = requests.get(href)
        res.encoding = 'utf-8'
        soup = BeautifulSoup(res.text,'html.parser')
        general = soup.select('.miaoshu-ul > .w50')
        
        register = general.select('s2')
        register =register[0].text if register else 'Error'
        
        
        print(register,href)
        info.append({'title': title,'area(m²)': area,'totalPrice(元)': totalPrice,'unitPrice(元 m²/day)': unitPrice, 'loc': loc})  
# pandas.DataFrame(info , columns=['title', 'area(m²)','loc','unitPrice(元 m²/day)','totalPrice(元)'])
info = sorted(info, key = lambda x : x['totalPrice(元)'], reverse=True) 

pandas.DataFrame(info)
#df.plot(kind='bar',x = 'loc',y = 'totalPrice(元)')

all = pandas.DataFrame(info)
all.to_excel('news.xlsx')

33


In [19]:
# -*- coding: UTF-8 -*-
from urllib import request
from bs4 import BeautifulSoup
import pandas

prefix = 'https://www.haozu.com/sh/'
local = ['house-pudongxinqu','house-huangpu','house-xuhui','house-changning','house-jingan','house-putuo','house-zhabei'
          ,'house-hongkou','house-yangpu','house-minhang','house-baoshan','house-jiading','house-songjiang'
          ,'house-qingpu','house-fengxian','house-shanghaizhoubian'] 
house1 = [prefix + item + '/a1/' for item in local] # 普通办公
house2 = [prefix + item + '/a6/' for item in local] # 创意园区

# 1 根据房子的类型，获取此类型下的每一页地址
def getFirstUrl(prefix):
    allUrl = [prefix]
    for item in range(2, 3) : 
        temp = str(prefix) + 'o' + str(item) + '/'
        allUrl.append(temp)
    return allUrl

# 2 根据这个类型下的所有页面地址,获取每一个房子的具体地址
def getSecondUrl(prefixList):
    all = [] # 保存每一个房源的地址，嵌套着字典
    hrefs = [] # 保存每一个房源的地址
    req = getRequest()
    for url in prefixList:
        response = req.urlopen(url)
        html = response.read().decode("utf-8")
        soup = BeautifulSoup(html)
        content = soup.select('.list-content')
        if len(content) !> 0:
            break
        href = [ url + item.select('a[style=""]')[0]['href'] for item in content]
        # print(href)
        hrefs.append(href)
    # 将所有地址保存进excel
    hrefs = sum(hrefs,[])
    for item in hrefs:
        all.append({'href1':item})
    all = pandas.DataFrame(all)
    all.to_excel('a1.xlsx')
    return hrefs

def getDetai():
    pass

# 获取设置过ua 和 ip 的request
def getRequest():
    proxy = {'https':'119.101.117.215:9999'}
    proxy_support = request.ProxyHandler(proxy)
    opener = request.build_opener(proxy_support)
    opener.addheaders = [('User-Agent','Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36')]
    request.install_opener(opener)
    return request

def mymain():
    allPage = getFirstUrl(house1[0])
    print("allPage:",allPage)
    allUrl = getSecondUrl(allPage)
    print(allUrl)
mymain()
# req = getRequest()
# response = req.urlopen('https://www.haozu.com/sh/house-pudongxinqu/a1/o2/')
# html = response.read().decode("utf-8")
# print(html)

# if __name__ == "__main__":
#     #访问的网址
#     url = 'https://www.haozu.com/sh/house631419/'
#     proxy = {'https':'119.101.117.215:9999'}
#     proxy_support = request.ProxyHandler(proxy)
#     opener = request.build_opener(proxy_support)
#     opener.addheaders = [('User-Agent','Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36')]
#     request.install_opener(opener)
#     response = request.urlopen(url)
#     html = response.read().decode("utf-8")
#     print(html)
#     soup = BeautifulSoup(html)
#     a = soup.select('.detail-box.v3-detail-box.clearfix')
#     b = soup.select('.warp.clearfix')[1]
#     allUrl = soup.select('a[style=""]')
    
# #     for item in allUrl:
# #         print(item['href'])
    
#     #打印信息
#     print(a,b)

allPage: ['https://www.haozu.com/sh/house-huangpu/a1/', 'https://www.haozu.com/sh/house-huangpu/a1/o2/']
['https://www.haozu.com/sh/house-huangpu/a1//sh/house1090560', 'https://www.haozu.com/sh/house-huangpu/a1//sh/house976735', 'https://www.haozu.com/sh/house-huangpu/a1//sh/house632793', 'https://www.haozu.com/sh/house-huangpu/a1//sh/house1090561', 'https://www.haozu.com/sh/house-huangpu/a1//sh/house1077332', 'https://www.haozu.com/sh/house-huangpu/a1//sh/house978337', 'https://www.haozu.com/sh/house-huangpu/a1//sh/house1180986', 'https://www.haozu.com/sh/house-huangpu/a1//sh/house262216', 'https://www.haozu.com/sh/house-huangpu/a1//sh/house262293', 'https://www.haozu.com/sh/house-huangpu/a1//sh/house614791', 'https://www.haozu.com/sh/house-huangpu/a1//sh/house1111246', 'https://www.haozu.com/sh/house-huangpu/a1//sh/house1070531', 'https://www.haozu.com/sh/house-huangpu/a1//sh/house1097836', 'https://www.haozu.com/sh/house-huangpu/a1//sh/house389682', 'https://www.haozu.com/sh/house-h

In [1]:
# from urllib import request
# url = 'https://www.haozu.com/sh/house-list/'
# proxy = {'https':'121.61.33.95:9999'}
# proxy_support = request.ProxyHandler(proxy)
# opener = request.build_opener(proxy_support)
# opener.addheaders = [("Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)")]
# request.install_opener(opener)
# print(url)
# response = request.urlopen(url)
# html = response.read().decode("utf-8")
# print(html)

In [23]:
user_agent = [
    "Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_8; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50",
    "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:38.0) Gecko/20100101 Firefox/38.0",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; .NET4.0C; .NET4.0E; .NET CLR 2.0.50727; .NET CLR 3.0.30729; .NET CLR 3.5.30729; InfoPath.3; rv:11.0) like Gecko",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)",
    "Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0)",
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1)"
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1",
    "Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/20100101 Firefox/4.0.1",
    "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; en) Presto/2.8.131 Version/11.11",
    "Opera/9.80 (Windows NT 6.1; U; en) Presto/2.8.131 Version/11.11",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_0) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Maxthon 2.0)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; TencentTraveler 4.0)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; The World)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SE 2.X MetaSr 1.0; SE 2.X MetaSr 1.0; .NET CLR 2.0.50727; SE 2.X MetaSr 1.0)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; 360SE)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Avant Browser)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)",
    "Mozilla/5.0 (iPhone; U; CPU iPhone OS 4_3_3 like Mac OS X; en-us) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8J2 Safari/6533.18.5",
    "Mozilla/5.0 (iPod; U; CPU iPhone OS 4_3_3 like Mac OS X; en-us) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8J2 Safari/6533.18.5",
    "Mozilla/5.0 (iPad; U; CPU OS 4_3_3 like Mac OS X; en-us) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8J2 Safari/6533.18.5",
    "Mozilla/5.0 (Linux; U; Android 2.3.7; en-us; Nexus One Build/FRF91) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1",
    "MQQBrowser/26 Mozilla/5.0 (Linux; U; Android 2.3.7; zh-cn; MB200 Build/GRJ22; CyanogenMod-7) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1",
    "Opera/9.80 (Android 2.3.4; Linux; Opera Mobi/build-1107180945; U; en-GB) Presto/2.8.149 Version/11.10",
    "Mozilla/5.0 (Linux; U; Android 3.0; en-us; Xoom Build/HRI39) AppleWebKit/534.13 (KHTML, like Gecko) Version/4.0 Safari/534.13",
    "Mozilla/5.0 (BlackBerry; U; BlackBerry 9800; en) AppleWebKit/534.1+ (KHTML, like Gecko) Version/6.0.0.337 Mobile Safari/534.1+",
    "Mozilla/5.0 (hp-tablet; Linux; hpwOS/3.0.0; U; en-US) AppleWebKit/534.6 (KHTML, like Gecko) wOSBrowser/233.70 Safari/534.6 TouchPad/1.0",
    "Mozilla/5.0 (SymbianOS/9.4; Series60/5.0 NokiaN97-1/20.0.019; Profile/MIDP-2.1 Configuration/CLDC-1.1) AppleWebKit/525 (KHTML, like Gecko) BrowserNG/7.1.18124",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows Phone OS 7.5; Trident/5.0; IEMobile/9.0; HTC; Titan)",
    "UCWEB7.0.2.37/28/999",
    "NOKIA5700/ UCWEB7.0.2.37/28/999",
    "Openwave/ UCWEB7.0.2.37/28/999",
    "Mozilla/4.0 (compatible; MSIE 6.0; ) Opera/UCWEB7.0.2.37/28/999",
    # iPhone 6：
    "Mozilla/6.0 (iPhone; CPU iPhone OS 8_0 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Version/8.0 Mobile/10A5376e Safari/8536.25",

]
user_agent[random.randint(0,len(user_agent))-1]

a
